In [2]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e

dic = {
    'TSS_low':'./data/gene/low_expressed.bed',
    'location':'./data/gene/GRCh37.gene.bed',
    'bam':'./data/051.bam',
    'test_bam':'./data/051_2.bam',
    'test_bam2':'./data/051_3.bam',
    'bam_123':'./data/051_123.bam',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed'
    }

chr_to_id = {
    'chr1':'1','chr2':'2','chr3':'3','chr4':'4','chr5':'5','chr6':'6','chr7':'7','chr8':'8','chr9':'9','chr10':'10','chr11':'11','chr12':'12','chr13':'13','chr14':'14','chr15':'15','chr16':'16','chr17':'17','chr18':'18','chr19':'19','chr20':'20','chr21':'21','chr22':'22','chrX':'X','chrY':'Y'
}

up = 1000
down = 1000

bamfile = ps.AlignmentFile(dic['bam'],'rb')
testbam = ps.AlignmentFile(dic['test_bam'],'rb')
testbam2 = ps.AlignmentFile(dic['test_bam2'],'rb')
bam123 = ps.AlignmentFile(dic['bam_123'],'rb')

TSS_low = []
with open(dic['TSS_low'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['chr1','chr2','chr3']:
            TSS_low.append(TSS(chr_to_id[ll[0]],int(ll[1])))

TSS_silent = []
with open(dic['TSS_silent'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['1','2','3']:
            TSS_silent.append(TSS(ll[0], int(ll[1])+int(1000)))
# TSS_silent2 = []
# with open(dic['TSS_silent'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         if ll[0] == '2':
#             TSS_silent.append(TSS(ll[0], int(ll[1])+int(1000)))
TSS_HK = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '1':
            TSS_HK.append(TSS(ll[0], int(ll[1])+int(1000)))

test_HK = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '2':
            test_HK.append(TSS(ll[0], int(ll[1])+int(1000)))
test_HK2 = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '3':
            test_HK2.append(TSS(ll[0], int(ll[1])+int(1000)))

In [3]:
TSS_HK = TSS_HK + test_HK + test_HK2
long_array = np.zeros((len(TSS_HK), int(up+down)), dtype=float)
short_array = np.zeros((len(TSS_HK), int(up+down)), dtype=float)
for j, tss in enumerate(TSS_HK):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    
    plist = []
    for r in bam123.fetch(chrom, start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            ss = max(0, r.reference_start - start)
            ee = min(r.reference_start - start + abs(r.isize), end - start)
            if r.isize <= 150:
                # print("!!!!")
                for i in range(ss, ee):
                    short_array[j][i] += 1
            if r.isize > 150:
                for i in range(ss, ee):
                    long_array[j][i] += 1

HK_mean = []
HK_std = []
for i in range(up+down):
    HK_mean.append(np.mean([long_array[j,i] + short_array[j,i] for j in range(len(TSS_HK))]))
    HK_std.append(np.std([long_array[j,i] + short_array[j,i] for j in range(len(TSS_HK))], ddof=1))


In [ ]:
# print(len(TSS_HK))
# for i in range(995,1005):
#     qq = []
#     for j in range(len(TSS_HK)):
#         qq.append(long_array[j][i] + short_array[j][i])
#     dd = {}
#     for q in sorted(qq):
#         if q in dd:
#             dd[q] += 1/len(qq)
#         else:
#             dd[q] = 1/len(qq)
#     x = [ke for ke in dd]
#     y = [dd[ke] for ke in dd]
#     # print(kstest(y,'norm'))
#     # print(normaltest(y))
#     plt.bar([ke for ke in dd], [dd[ke] for ke in dd])
#     plt.show()

In [4]:
TSS_silent = TSS_silent
long_array = np.zeros((len(TSS_silent), int(up+down)), dtype=float)
short_array = np.zeros((len(TSS_silent), int(up+down)), dtype=float)
for j, tss in enumerate(TSS_silent):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    
    plist = []
    for r in bam123.fetch(chrom, start-500, end + 500):
         if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            ss = max(0, r.reference_start - start)
            ee = min(r.reference_start - start + abs(r.isize), end - start)
            if r.isize <= 150:
                # print("!!!!")
                for i in range(ss, ee):
                    short_array[j][i] += 1
            if r.isize > 150:
                for i in range(ss, ee):
                    long_array[j][i] += 1

silent_mean = []
silent_std = []
for i in range(up+down):
    silent_mean.append(np.mean([long_array[j,i] + short_array[j,i] for j in range(len(TSS_silent))]))
    silent_std.append(np.std([long_array[j,i] + short_array[j,i] for j in range(len(TSS_silent))], ddof=1))

In [ ]:
# print(len(TSS_silent))
# for i in range(995,1005):
#     qq = []
#     for j in range(len(TSS_silent)):
#         qq.append(long_array[j][i] + short_array[j][i])
#     dd = {}
#     for q in sorted(qq):
#         if q in dd:
#             dd[q] += 1/len(qq)
#         else:
#             dd[q] = 1/len(qq)
#     x = [ke for ke in dd]
#     y = [dd[ke] for ke in dd]
#     plt.bar(x,y)
#     plt.show()
#     print(normaltest(y))

In [28]:
for j, tss in enumerate(TSS_silent[:10]):
    print('-------------',j,'---------------')
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    plist = []
    short_array = np.zeros(int(up+down), dtype=float)
    long_array = np.zeros(int(up+down), dtype=float)
    for r in bamfile.fetch(chrom, start-500, end + 500):
         if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            ss = max(0, r.reference_start - start)
            ee = min(r.reference_start - start + abs(r.isize), end - start)
            if r.isize <= 150:
                for i in range(ss, ee):
                    short_array[i] += 1
            if r.isize > 150:
                for i in range(ss, ee):
                    long_array[i] += 1
    fr = []
    for i in range(950,1050):
        u1 = HK_mean[i]
        u2 = silent_mean[i]
        # s1 = 1
        # s2 = 2
        s1 = HK_std[i]
        s2 = silent_std[i]

        x = short_array[i] + long_array[i]
        # print(x,u1,u2)
        # continue
        t = sympy.Symbol('t')
        mu = t * u1 + (1-t) * u2
        sig = sympy.sqrt(t**2 * s1**2 + (1-t)**2 * s2**2)
        y = -1*sympy.log(math.sqrt(2*math.pi)*sig) - (x - mu)**2 / (2*sig**2) + sympy.log(t)
        gg = float('-inf')
        kk = 0
        ttt = 0
        while kk <= 1:
            re = y.evalf(subs={t:kk})
            if gg < re:
                gg = re
                ttt = kk
            kk += 0.01
        fr.append(ttt)
    print(np.mean(fr))
        # print(re,ttt,x,u1,u2,s1,s2)
        # fr.append(ttt)
        # print(gg, ttt)
        # print("y=",y)
        # y_ = sympy.diff(y,t)
        # print("y_=",y_)
        # ttt = sympy.solve(y_,t)
        # print("ttt=",ttt)
        # ttt[1] = abs(ttt[1])
        # if ttt[1] > 1:
        #     fr.append(1)
        # elif ttt[1] < 0:
        #     fr.append(0)
        # else:
        #     fr.append(ttt[1])
    # print(fr,np.mean(fr))


------------- 0 ---------------
0.9900000000000001
------------- 1 ---------------
0.9900000000000001
------------- 2 ---------------
0.7367000000000005
------------- 3 ---------------
0.7041000000000003
------------- 4 ---------------
0.3627000000000001
------------- 5 ---------------
0.25700000000000006
------------- 6 ---------------
0.19990000000000002
------------- 7 ---------------
0.8148000000000002
------------- 8 ---------------
0.27350000000000013
------------- 9 ---------------
0.2617000000000001
